In [15]:
# import pyshark
# import requests
# import threading

# def packet_callback(pkt, target_ip):
#     if pkt.highest_layer in ['TCP', 'DNS', 'HTTP', 'SSL', 'TLS']:
#         # Check if the packet's source or destination address matches the target IP
#         if hasattr(pkt, 'ip') and (pkt.ip.src == target_ip or pkt.ip.dst == target_ip):
#             print(pkt)

# def resolve_domain_to_ip(domain):
#     try:
#         ip_address = socket.gethostbyname(domain)
#         return ip_address
#     except socket.gaierror:
#         print("Error: Unable to resolve the domain name to an IP address.")
#         return None

# def capture_packets(target_ip):
#     capture = pyshark.LiveCapture(interface='en0', display_filter='tcp or dns or http or ssl or tls')
#     for packet in capture.sniff_continuously(packet_count=10):
#         packet_callback(packet, target_ip)

# def generate_traffic(url):
#     response = requests.get(url, verify=False)
#     print("Generated traffic to:", url)

# if __name__ == "__main__":
#     target_domain = 'www.indiatimes.com'
#     target_ip = resolve_domain_to_ip(target_domain)

#     if target_ip:
#         # Start capturing packets in a separate thread
#         capture_thread = threading.Thread(target=capture_packets, args=(target_ip,))
#         capture_thread.start()

#         # Generate traffic to the target IP address
#         generate_traffic('https://' + target_ip)

#         # Wait for the capture thread to finish
#         capture_thread.join()
#     else:
#         print("Exiting due to domain resolution error.")

import pyshark
import pandas as pd
import threading

def packet_to_dict(pkt):
    packet_dict = {
        'time': pkt.sniff_time.timestamp(),
        'protocol': pkt.highest_layer,
        'source': pkt.ip.src if hasattr(pkt, 'ip') else None,
        'destination': pkt.ip.dst if hasattr(pkt, 'ip') else None,
        'length': int(pkt.length),
        # Add more fields as needed
    }
    return packet_dict

def packet_callback(pkt, packets_df):
    if isinstance(packets_df, pd.DataFrame):
        packet_dict = packet_to_dict(pkt)
        return pd.concat([packets_df, pd.DataFrame([packet_dict])], ignore_index=True)
    else:
        print("Error: packets_df is not a DataFrame.")
        return None

def capture_packets():
    packets_df = pd.DataFrame(columns=['time', 'protocol', 'source', 'destination', 'length'])
    # Start capturing packets
    capture = pyshark.LiveCapture(interface='en0', display_filter='tcp or dns or http or ssl or tls')
    capture.sniff(timeout=10)

    # Iterate over captured packets and store them in DataFrame
    for packet in capture.sniff_continuously(packet_count=10):
        packets_df = packet_callback(packet, packets_df)
        if packets_df is None:
            break  # Stop capturing if DataFrame becomes invalid

    print(packets_df)

if __name__ == "__main__":
    # Create a separate thread for packet capture
    capture_thread = threading.Thread(target=capture_packets)
    capture_thread.start()
    capture_thread.join()




/var/folders/_v/2_7ylzwd1r140fnllwbxf9hr0000gn/T/ipykernel_17985/3698867670.py:63: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([packets_df, pd.DataFrame([packet_dict])], ignore_index=True)


           time protocol         source    destination length
0  1.711084e+09      TCP    172.20.10.2  17.248.209.73     78
1  1.711084e+09      TCP  17.248.209.73    172.20.10.2     66
2  1.711084e+09      TCP    172.20.10.2  17.248.209.73     54
3  1.711084e+09      TLS    172.20.10.2  17.248.209.73    571
4  1.711084e+09      TCP  17.248.209.73    172.20.10.2     54
5  1.711084e+09      TLS  17.248.209.73    172.20.10.2   1414
6  1.711084e+09      TCP  17.248.209.73    172.20.10.2   1414
7  1.711084e+09      TCP  17.248.209.73    172.20.10.2   1414
8  1.711084e+09      TCP  17.248.209.73    172.20.10.2     70
9  1.711084e+09      TCP  17.248.209.73    172.20.10.2   1414
